# Simulation counterbalancing vs. regression
To do:
- create correlated data (perhaps in separate function?)
- 

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_classif
from skbold.preproc import ConfoundRegressor, MajorityUndersampler
%matplotlib inline

In [235]:
iters = 5
n_samp = 100
n_feat = 5
n_fold = 10
std = 1

skf = StratifiedKFold(n_splits=n_fold)

results = {'accuracy': np.zeros(iters)}

confound_control = 'regress'
acc = np.zeros((iters, n_fold))
for i in range(iters):
    
    if i % int(iters / 5) == 0:
        print("Iteration %i" % i)

    n_half = int(n_samp / 2)
    y = np.repeat([0, 1], repeats=n_half)
    c = np.roll(y, 10)
    
    data = np.random.randn(n_samp, n_feat)
    #data[c == 1, :] += 1
    data[y == 1, :] += 0.5
    X = data
    
    for ii, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        
        pipeline = [
            ('confoundreg', ConfoundRegressor(confound=y, fit_idx=train_idx, cross_validate=True)),
            ('scaler', StandardScaler()),
            ('svm', SVC(kernel='linear'))
        ]
        
        if confound_control != 'regress':
            pipeline.pop(0)
        
        pipeline = Pipeline(pipeline)
        
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        if confound_control == 'cb':
            idx = counterbalance(y_train, c[train_idx], verbose=False)
            X_train, y_train = X_train[idx], y_train[idx]
            
        #print(f_classif(X_train, y_train)[0])
        pipeline.fit(X_train, y_train)
        acc[i, ii] = pipeline.score(X_test, y_test)
    results['accuracy'][i] = acc[i, :].mean()
print(np.mean(results['accuracy']))

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
0.496


In [355]:
from copy import deepcopy
from scipy.stats import itemfreq

class CounterbalancedStratifiedSplit(object):
    
    def __init__(self, X, y, c, counterbalance_tolerance=.05, n_splits=5, verbose=False):
        self.X = X
        self.y = y
        self.c = c
        self.counterbalance_tolerance = counterbalance_tolerance
        self.n_splits = n_splits
        self.verbose = verbose

        self.subsample_idx = np.arange(len(y))
        self.c_classes = np.unique(c)
        self.n_confound_classes = len(self.c_classes)
        self.counterbalanced_proportion = np.divide(1, self.n_confound_classes)
        self.y_classes = np.unique(y)

        self.seed = None
        self.checked_possible = False
    
    def check_possible(self):
        """ Check if the confound classes are counterbalanced in each class of y """
        
        if self.verbose:
            print('Checking if a counterbalanced split is possible without subsampling...')
        
        select_idx = np.array([])
        for y_class in self.y_classes:
            idx = y == y_class
            y_sub = y[idx]
            
            # Get frequency table of all confound classes
            freqs = itemfreq(c[idx])
            
            # Get absolute difference
            diffs = freqs[:,1].reshape(-1, 1) - freqs[:,1]
            
            # If any absolute difference is larger than 0, we have confound imbalance
            if np.max(diffs) > 0:
                print('Oops! The confound classes are not counterbalanced within y-class %s. Subsampling...' % str(y_class))
                
                for c_class in self.c_classes:
                    # From self.subsample_idx, sample the idx of the number of y-class observations necessary to obtain a balanced class
                    select_idx = np.concatenate((select_idx, np.random.choice(self.subsample_idx[(y == y_class) & (c == c_class)], size=np.min(freqs[:,1]), replace=False)))
            else:
                # If confound classes are counterbalanced within this y_class, select all of these idx
                select_idx = np.concatenate((select_idx, self.subsample_idx[idx]))

        self.checked_possible = True
        self.subsample_idx = np.sort(select_idx).astype(int)
        
        if self.verbose:
            print('After subsampling, proportions of c-classes in y_classes:')
            y_tmp = y[self.subsample_idx]
            c_tmp = c[self.subsample_idx]

            for y_class in self.y_classes:
                print(itemfreq(c_tmp[y_tmp == y_class]))

    
    def find_counterbalanced_seed(self, max_attempts=50000):
        """ Find a seed of Stratified K-Fold that gives counterbalanced classes """

        if not self.checked_possible:
            self.check_possible()
        
        X_tmp = deepcopy(self.X)[self.subsample_idx]
        y_tmp = deepcopy(self.y)[self.subsample_idx]
        c_tmp = deepcopy(self.c)[self.subsample_idx]
                
        for i, sd in enumerate(np.random.randint(low=0, high=1e7, size=max_attempts, dtype=int)):
            skf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=sd)
            bad_split = False

            for (train_idx, test_idx) in skf.split(X_tmp, y_tmp):
                if bad_split:
                    if self.verbose and i % 100 == 0:
                        print('.', end='')
                    break

                for set_ in (train_idx, test_idx):
                    y_set = y_tmp[set_]
                    c_set = c_tmp[set_]
                    
                    for y_class in self.y_classes:
                        y_set_class = y_set[y_set == y_class]
                        c_set_class = c_set[y_set == y_class]

                        # Get proportions of each class of c for each y_class
                        proportions = itemfreq(c_set_class).astype(float)[:,1]/len(c_set_class)

                        # Check whether the proportions are equal
                        if proportions.shape[0] == 1 or np.max(proportions.reshape(-1,1) - proportions) > self.counterbalance_tolerance:
                            bad_split = True

            if not bad_split:
                if self.verbose:
                    print('\nGood split found! Seed %d was used.' % i)
                self.seed = sd
                return
                
        if self.seed is None:
            raise(ValueError('\nSorry, could not find any good split...'))
            
        
    def split(self, X, y):
        """ The final idx to output are subsamples of the subsample_idx... """
        
        if self.seed is None:
            raise(IOError('You need to run CounterbalancedStratifiedSplit.find_counterbalanced_seed() first'))
        
        X_tmp = X[self.subsample_idx]
        y_tmp = y[self.subsample_idx]
        
        skf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.seed)
        
        for (train_idx, test_idx) in skf.split(X=X_tmp, y=y_tmp):
            yield ((self.subsample_idx[train_idx], self.subsample_idx[test_idx]))

### Test the CSS-class

In [375]:
css = CounterbalancedStratifiedSplit(X, y, c, n_splits=5, verbose=True)
css.check_possible()
y_tmp = y[css.subsample_idx]
c_tmp = c[css.subsample_idx]
print(np.hstack((y_tmp.reshape(len(y_tmp),1), c_tmp.reshape(len(c_tmp),1))))

css.find_counterbalanced_seed()
splts = css.split(X, y)

for spltn in splts:
    for i, set_ in enumerate(spltn):
        set_type = ['train', 'test'][i]
        print('%s set, y and confound:' % set_type)
        print(pd.DataFrame({'y': y[set_], 'confound': c[set_]}))
        
        print('\n%s set confound count per class: ' % set_type)
        print(itemfreq(c[set_]))

Checking if a counterbalanced split is possible without subsampling...
Oops! The confound classes are not counterbalanced within y-class 0. Subsampling...
Oops! The confound classes are not counterbalanced within y-class 1. Subsampling...
After subsampling, proportions of c-classes in y_classes:
[[ 0 10]
 [ 1 10]]
[[ 0 10]
 [ 1 10]]
[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]]
.....
Good split found! Seed 436 was used.
train set, y and confound:
    confound  y
0          1  0
1          1  0
2          1  0
3          1  0
4          1  0
5          1  0
6          1  0
7          1  0
8          0  0
9          0  0
10         0  0
11         0  0
12         0  0
13         0  0
14         0  0
15         0  0
16         0  1
17         0  1
18        